In [1]:
# 2. GRADIENT BOOSTING
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import pickle


In [2]:
X_train = pd.read_csv("../../data/processed/X_train.csv")
y_train = pd.read_csv("../../data/processed/y_train.csv")["RUL_steps"].values
X_val = pd.read_csv("../../data/processed/X_val.csv")
y_val = pd.read_csv("../../data/processed/y_val.csv")["RUL_steps"].values
train_balanced =  pd.read_csv("../../data/processed/train_balanced.csv")
train_df = pd.read_csv("../../data/processed/train_df.csv")
val_df = pd.read_csv("../../data/processed/val_df.csv")

In [ ]:
weights = train_balanced['risk_bin'].map({
    'MUY_CRITICO': 8.0,
    'CRITICO': 5.0, 
    'ALTO_RIESGO': 2.0,
    'BAJO_RIESGO': 1.0
}).values
# 4. GRADIENT BOOSTING (NO necesita escalado)
gb_model = GradientBoostingRegressor(n_estimators=500, max_depth=6, learning_rate=0.05, subsample=0.8,
                                    min_samples_split=10, min_samples_leaf=5, random_state=42)
gb_model.fit(X_train, y_train, sample_weight=weights)
gb_pred = gb_model.predict(X_val)
# MÉTRICAS GRADIENT BOOSTING (CRÍTICOS RUL<50)
mask_critica = y_val < 50
gb_mae_total = mean_absolute_error(y_val, gb_pred)
gb_mae_criticos = mean_absolute_error(y_val[mask_critica], gb_pred[mask_critica])
gb_r2_total = r2_score(y_val, gb_pred)

print(f"📊 GRADIENT BOOSTING:")
print(f"MAE total:     {gb_mae_total:.2f}")
print(f"MAE RUL<50:    {gb_mae_criticos:.2f}")
print(f"R² total:      {gb_r2_total:.4f}")
print(f"Nº críticos:   {mask_critica.sum():,} ({mask_critica.mean()*100:.1f}%)")
print()




### Pickle

In [ ]:
with open("../../models/gradient_boost.pkl", "wb") as f:
    pickle.dump(gb_model, f)